Basic Trading Operation
=============

This notebook implements trading operations on ANX exchange, like authentication, visualization of series and results and the actual trading. Implementation of algorithms should be done in separate modules which can then be imported here.

In [15]:
import base64, hashlib, hmac, time, urllib, json
from urllib.request import urlopen
from getpass import getpass, getuser
import requests
base = 'https://anxpro.com/'
keys = json.load(open('keys.json')) # This file must be created on disk with you authentication credentials

Basic connection class
-------------------------

Below we define a basic connection class so that we can talk to the API, and test it by asking for the my order list.

In [25]:
def post_request(key, secret, path, data):
    hmac_obj = hmac.new(secret, (path + chr(0) + data).encode('utf-8'), hashlib.sha512)
    hmac_sign = base64.b64encode(hmac_obj.digest())

    header = {
        'Content-Type': 'application/json',
        'User-Agent': 'anxv2 based client',
        'Rest-Key': key,
        'Rest-Sign': hmac_sign,
    }

    response = requests.post(base + path, headers=header, data=data)
    return response.json()


def gen_tonce():
    return str(int(time.time() * 1e6))


class ANX:

    def __init__(self, key, secret):
        self.key = key
        self.secret = base64.b64decode(secret)

    def request(self, path, params={}):
        params = dict(params)
        params['tonce'] = gen_tonce()
        data = json.dumps(params)

        result = post_request(self.key, self.secret, path, data)
        
        if result['resultCode'] == 'OK':
            return result
        else:
            raise Exception(result['resultCode'])

X = ANX(keys['ANX']['key'], keys['ANX']['secret'])
result = X.request('api/3/order/list')
result

{'count': 0, 'orders': [], 'resultCode': 'OK', 'timestamp': 1452081334423}

Getting an API Key
---------------------
Here we will try to generate and get an API key directly through the api. Alternatively, You can generate it directly through the web after logging in to your account at www.anxpro.com.

In [27]:
user = input("enter your email:")
password = getpass("Enter your password:")

enter your email:fccoelho@gmail.com
Enter your password:········


In [28]:
#not working, ignore
params = {
    'nonce': str(int(time.time() * 1e6)),
    'username': user,
    'password': password,
    'otp': "400345", #fill in from your cell phone,
    'deviceId': 'vaio'
} 
resp = X.request('api/3/apiKey', params)

resp

Exception: UNAUTHORISED

In [18]:
hmac.new?